In [1]:
import glob
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.filters import threshold_otsu
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [2]:
with open("VOCNACD/ImageSets/Main/train.txt") as f:
    filenames = f.readlines()
filenames = [x.strip() for x in filenames]

for filename in filenames:
    annotation_path = os.path.join('VOCNACD', 'Annotations',filename+'.xml')
    image_path = os.path.join('VOCNACD', 'JPEGImages',filename+'.jpg')
    img = cv2.imread(image_path)
    os.makedirs('clusters_training/'+filename)
    root = ET.parse(annotation_path).getroot()
    i = 0
    for obj in root.iter('object'):
        xml_box = obj.find('bndbox')
        xmin = int(xml_box.find('xmin').text)
        ymin = int(xml_box.find('ymin').text)
        xmax = int(xml_box.find('xmax').text)
        ymax = int(xml_box.find('ymax').text)
        crop = img[ymin:ymax,xmin:xmax]
        crop_path = os.path.join('clusters_training',filename,filename+'_'+str(i).zfill(3)+'.jpg')
        cv2.imwrite(crop_path,crop)
        i = i + 1

IOError: [Errno 2] No such file or directory: 'VOCNACD/Annotations/NFS_d___01754_00008 1.xml'